# Deep Learning Black-Scholes model

This is a simple example of learning the valuation function of a plain-vanilla put option in 5 dimensions:
* Spot price
* Time to maturity
* Volatility
* Discount rate
* Dividend rate

Due to the obvious scalability of the model, the Strike price is fixed at $1.
Using this code, the user should be able to achieve the average accuracy of about 0.1 cent. 

The network architecture is a fully connected MLP with four hidden layers with 100 ReLU neurons each.

The code will detect and run on GPU (if available) or CPU.

In [ ]:
import numpy as np
from scipy.stats import norm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

## Define Black Scholes put model

In [ ]:
def ds(K, S, T, vol, r, q):
    vol_T = vol * np.sqrt(T)
    d1 = (np.log(S/K) + (r - q + 0.5 * vol * vol) * T) / vol_T
    d2 = d1 - vol_T
    return d1, d2

def put(K, S, T, vol, r, q):

    disc = np.exp(-r * T)
    pv_K = K * disc
    spot_after_div = S * np.exp(-q * T)

    d1, d2 = ds(K, S, T, vol, r, q)
    v = norm.cdf(-d2) * pv_K - norm.cdf(-d1) * spot_after_div
    return v * 100.

v_put = np.vectorize(put)

## Generate train, test, and validation data

In [ ]:
# Fix seeds for reproducibility
seed = 314
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# Training domain
domain = {
    "spot": (0.5, 2),
    "time": (0, 3.0),
    "sigma": (0.1, 0.5),
    "rate": (-0.01, 0.03),
    "div": (0, 0.02)
}

In [ ]:
n_samples = 100000          # Total number of samples
pct_test = 0.2              # Portion for test set
pct_validation = 0.1        # Portion for validation set

samples = np.zeros(shape=(len(domain.keys()), n_samples))
for i, r in enumerate(domain.values()):
    samples[i] = np.random.uniform(r[0], r[1], n_samples)

In [ ]:
#### Calculate BSM values
values = v_put(K=1, S=samples[0], T=samples[1], vol=samples[2], r=samples[3], q=samples[4])

In [ ]:
samples_t = torch.from_numpy(samples.T).float()
values_t = torch.from_numpy(values).float().unsqueeze(dim=1)

## Define DNN architecture and learning function

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_layers, n_output):
        super(Net, self).__init__()

        self.n_hidden = n_hidden
        self.n_layers = n_layers

        self.linears = torch.nn.ModuleList([torch.nn.Linear(n_feature, n_hidden)])
        self.linears.extend([torch.nn.Linear(n_hidden, n_hidden) for i in range(1, n_layers)])
        self.linears.append(torch.nn.Linear(n_hidden, n_output))

    def forward(self, x):
        for lin in self.linears[:-1]:
            x = F.relu(lin(x))       # Activation function for hidden layer
        x = self.linears[-1](x)             # Apply last layer without activation
        return x

In [ ]:
def fit_net(net: Net, n_epochs: int, x: torch.tensor, y: torch.tensor,
            pct_test: int, pct_validation: int, device: str='cpu'):

    n = y.size()[0]
    n_train = int(np.round(n * (1 - pct_test - pct_validation)))
    n_test = int(np.round(n * pct_test))
    
    x_train = x[:n_train]
    x_test = x[n_train:(n_train + n_test)]
    y_train = y[:n_train]
    y_test = y[n_train:(n_train + n_test)]
    
    net.to(device)
    x_ = x_train.to(device)
    y_ = y_train.to(device)

    x_test_ = x_test.to(device)
    y_test_ = y_test.to(device)

    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    loss_func = torch.nn.MSELoss()
   
    l = 123.45
    best_l = 1e-3
    checkpoint = {}

    
    for e in range(n_epochs):
        prediction = net(x_)
        loss = loss_func(prediction, y_)

        prediction_test = net(x_test_)
        loss_test = loss_func(prediction_test, y_test_)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        l = loss_test.data.cpu().numpy()
        if l.item() < best_l:
            best_l = l.item()
            checkpoint = {
                "n_hidden": net.n_hidden,
                "n_layers": net.n_layers,
                "model_state_dict": net.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            }
        if (e + 1) % 100 == 0:
            print(f"\tEpoch: {e+1}\tL2 Loss = {loss.data.cpu().numpy()}")

    return best_l, checkpoint

## Determine device

In [ ]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    print(f"GPU detected. Running on {device}")
else:
    print("No GPU detected. Running on CPU")

## Define network architecture and train

In [ ]:
net = Net(n_feature=5, n_hidden=100, n_layers=4, n_output=1)  # define the network
print(net)  # net architecture

In [ ]:
n_epochs = 6000
ls, checkpoint = fit_net(net, n_epochs, samples_t, values_t, pct_test, pct_validation, device)
print(f"Best loss ={ls}")

## Make copy of network from checkpoint and validate results

In [ ]:
model = Net(n_feature=5,
            n_hidden=checkpoint["n_hidden"],
            n_layers=checkpoint["n_layers"],
            n_output=1)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model.to(device)

In [ ]:
n = values_t.size()[0]
ind_validation = n_train = int(np.round(n * (1 - pct_validation)))
samples_validation = samples_t[ind_validation:].to(device)
values_validation = values_t[ind_validation:]

In [ ]:
v_nn = model(samples_validation).flatten().data.cpu().numpy()
error = v_nn - values_validation.flatten().data.cpu().numpy()
mean_err = np.mean(error)
std_error = np.std(error)

In [ ]:
plt.hist(error, bins=50)
plt.title("Error histogram for validation set")
print(f"Mean error = {mean_err:.4f}, StDev = {std_error:.4f}")